#  Automated ML with azureml

The dependencies are imported

In [ ]:
import os
from azureml.core import Dataset, Datastore, Workspace
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

## Dataset

### Overview

We will try to predict the rating of modified version of the **Kaggle Trip advisor dataset**.

The Dataset contains a Trip Advisor hotel review text column as well as a Rating column with Ratings from 0 - 5 stars. 

> The Tripadvisor Hotel Review Dataset file, is derived from the publication: 
>
>_Alam, M. H., Ryu, W.-J., Lee, S., 2016. Joint multi-grain topic senti- ment: modeling semantic aspects for online >reviews. Information Sciences 339, 206–223._ 
>
> You can download the Dataset with the link:
> [trip-advisor-hotel-reviews](https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews)

In the original Dataset the target **Rating** column contains the values 0* - 5*.

In a modified version of the dataset we will try to predict the **norm_rating** column based on the **Review** text column as a **classification task** with:

* class 0 - Negative reviews (1* & 2* rating)
* class 1 - Neutral reviews (3* rating)
* class 2 - Positive reviews (4* & 5* rating)


## Initialize the Workspace and create an Experiment

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_review_classifier'

experiment=Experiment(ws, experiment_name)
experiment

## Load the Dataset and perform a train test split

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

filepath_2_dataset = r"C:\Users\christoph.hiemenz\Desktop\Grid_search_results\hotel_reviews_featurized_roberta.csv"
# Read the Dataset as a pandas dataframe
hotel_review_dataset = pd.read_csv(filepath_2_dataset)

### First the same train test split is performed for the Dataset to make it available to both AutoML and Hyperdrive

In [24]:
# Get hotel review text and normalized rating
X = hotel_review_dataset.drop(columns=['text'])
y = list(hotel_review_dataset.norm_rating)
X_train, X_test, y_train, y_test = train_test_split(hotel_review_dataset, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}\nX_test: {X_test.shape}\ny_train: {len(y_train)}\ny_test: {len(y_test)}")

X_train: (16392, 808)
X_test: (4099, 808)
y_train: 16392
y_test: 4099


### The training set and test sets will be registered separately to ensure strict separation

In [26]:
X_train['norm_rating'] = y_train
X_test['norm_rating'] = y_test 

<ipython-input-26-ba8989840918>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['norm_rating'] = y_train
<ipython-input-26-ba8989840918>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['norm_rating'] = y_test


### The AutoML train/testsets should contain just the text column and norm rating column (no feature engineering)

#### Upload the different train/test sets

In [29]:
X_train_automl = X_train.loc[:, ['text', 'norm_rating']]
X_test_automl = X_test.loc[:, ['text', 'norm_rating']]

os.makedirs("data", exist_ok=True)

# Upload the training/test data in the default datastore
train_dataset_path_automl = "data/train_set_automl.csv"
X_train_automl.to_csv(train_dataset_path_automl)
test_dataset_path_automl = "data/test_set_automl.csv"
X_test_automl.to_csv(test_dataset_path_automl)

train_dataset_path = "data/train_set.csv"
X_train.to_csv(train_dataset_path)
test_dataset_path = "data/test_set.csv"
X_test.to_csv(test_dataset_path)

datastore = ws.get_default_datastore()
datastore.upload(src_dir="data", target_path="data")

10726    0
14919    2
19098    1
2450     2
960      2
        ..
1576     1
18714    2
12690    2
18095    2
11836    2
Name: norm_rating, Length: 4099, dtype: int64

### Load the training and test Datasets and register them

In [ ]:
dataset_training = Dataset.Tabular.from_delimited_files(path = [(datastore, ("data/train_set_automl.csv"))])
dataset_training = dataset_training.register(workspace=ws, name="auto-ml-training-data", description="Hotel Review AutoML Training Data")

dataset_test =  Dataset.Tabular.from_delimited_files(path = [(datastore, ("data/test_set_automl.csv"))])
dataset_test = dataset_training.register(workspace=ws, name="auto-ml-test-data", description="Hotel Review AutoML Test Data")

## Define a Compute Target for AutoML

In [ ]:
## Define a Compute Target for AutoML
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-1"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing Compute Target")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## AutoML Configuration

* _experiment_timeout_minutes_: was set to prevent the experiment from running for long timer periods with high cost
* _max_concurrent_iterations_: was set to 4 since only 4 compute target nodes are available for paralle child runs
* _primary_metric_: was set to AUC_weighted since this includes a balance between false positive and true positive rate
* _n_cross_validations_: 5 crossvalidations were selected, since this results in a more robust mean/std estimation for each model

* _enable_early_stopping_: to prevent unproductive runs which lead to no improvement and costs
* _compute_target_: needs to be define to perform the AutoML computations
* _task_: needs to be classification since the label column is defining separate classes
* _training_data_: corresponds to the training set
* _label_column_: corresponds to the target/label column defining the separate classes
* _debug_log_: defined to enable detailed logging of automl errors

In [ ]:
## Define key AutoML Settings
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric": "AUC_weighted",
    "n_cross_validations": 5
}

## Setup an AutoMLConfig object
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task="classification",
    training_data=dataset_training,
    label_column_name="norm_rating",
    enable_early_stopping=True,
    debug_log="automl_errors.log",
    **automl_settings
)

In [ ]:
# The Experiment needs to be submitted in order to execute the AutoML run
automl_pipeline_run = experiment.submit(automl_config)

## Run Details

Write about the different models trained and their performance. Why do you think some models did better than others?

In [ ]:
from azureml.widgets import RunDetails
RunDetails(automl_pipeline_run).show()

In [ ]:
automl_pipeline_run.wait_for_completion(show_output=True)

## Performance metrics and Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.


### Print the performance metrics for the AutoML run

In [ ]:
metrics_output = automl_pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

### Get the best model and the best run

In [ ]:
best_run = automl_pipeline_run.get_best_run_by_primary_metric()
print(best_run.get_file_names())
best_model = best_run.register_model(workspace=ws, model_name="best-automl-model", model_path="outputs/automl_model.pkl")

In [ ]:
best_run, fitted_model = automl_pipeline_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
### Test the best model
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()

y_test = df_test['norm_rating']
X_test = df_test.drop(['norm_rating'], axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)
cm

## Model Deployment

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

service_name = 'automl-review-classification'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[best_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)
print("scoring URI: " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json
from azureml.core.authentication import InteractiveLoginAuthentication

# Get a token to authenticate to the compute instance from remote
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Create and submit a request using the auth header
headers = auth_header
# Add content type header
headers.update({'Content-Type':'application/json'})

# Sample data to send to the service
test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10],
    [10,9,8,7,6,5,4,3,2,1]
]})
test_sample = bytes(test_sample, encoding = 'utf8')

# Replace with the URL for your compute instance, as determined from the previous section
service_url = service.endpoint
# for a compute instance, the url would be https://vm-name-6789.northcentralus.instances.azureml.net/score
response = requests.post(service_url, test_sample, headers=headers)
print("prediction:", response.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(local_service.get_logs())

In [ ]:
mport os
import pickle
import json
import time
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from gensim.models.word2vec import Word2Vec

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)
SEQUENCE_LENGTH = 300

# Called when the deployed service starts
def init():
    global model
    global tokenizer
    global encoder
    global w2v_model

    # Get the path where the deployed model can be found.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), './models')
    # load models
    model = load_model(model_path + '/model.h5')
    w2v_model = Word2Vec.load(model_path + '/model.w2v')

    with open(model_path + '/tokenizer.pkl','rb') as handle:
        tokenizer = pickle.load(handle)

    with open(model_path + '/encoder.pkl','rb') as handle:
        encoder = pickle.load(handle)

# Handle requests to the service
def run(data):
    try:
        # Pick out the text property of the JSON request.
        # This expects a request in the form of {"text": "some text to score for sentiment"}
        data = json.loads(data)
        prediction = predict(data['text'])
        #Return prediction
        return prediction
    except Exception as e:
        error = str(e)
        return error

# Determine sentiment from score
def decode_sentiment(score, include_neutral=True):
    if include_neutral:
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE
        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

# Predict sentiment using the model
def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  